In [1]:
import numpy as np

In [2]:
import random

random.seed(11)

In [3]:
# import pandas library as pd
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/PhD UCO 062022/step by step/Datasets 12062022/cervical_cancer.csv")


In [ ]:
data_NaN = data.replace('?', pd.np.nan)

In [5]:
data_NaN = data_NaN.apply(pd.to_numeric, errors='ignore')

In [6]:
data_NaN = data_NaN.drop(columns=['STDs: Time since last diagnosis','STDs: Time since first diagnosis'], axis=1)

In [7]:
data_without_na = data_NaN.dropna()

In [8]:
data=data_without_na

In [9]:
data.reset_index(inplace = True)

In [10]:
data.head()

index  Age  Number of sexual partners  First sexual intercourse  \
0      0   18                        4.0                      15.0   
1      1   15                        1.0                      14.0   
2      3   52                        5.0                      16.0   
3      4   46                        3.0                      21.0   
4      5   42                        3.0                      23.0   

   Num of pregnancies  Smokes  Smokes (years)  Smokes (packs/year)  \
0                 1.0     0.0             0.0                  0.0   
1                 1.0     0.0             0.0                  0.0   
2                 4.0     1.0            37.0                 37.0   
3                 4.0     0.0             0.0                  0.0   
4                 2.0     0.0             0.0                  0.0   

   Hormonal Contraceptives  Hormonal Contraceptives (years)  ...  STDs:HPV  \
0                      0.0                              0.0  ...       0.0   
1                      0.0                              0.0  ...       0.0   
2                      1.0                              3.0  ...       0.0   
3                      1.0                             15.0  ...       0.0   
4                      0.0                              0.0  ...       0.0   

   STDs: Number of diagnosis  Dx:Cancer  Dx:CIN  Dx:HPV  Dx  Hinselmann  \
0                          0          0       0       0   0           0   
1                          0          0       0       0   0           0   
2                          0          1       0       1   0           0   
3                          0          0       0       0   0           0   
4                          0          0       0       0   0           0   

   Schiller  Citology  Biopsy  
0         0         0       0  
1         0         0       0  
2         0         0       0  
3         0         0       0  
4         0         0       0  

[5 rows x 35 columns]

In [11]:
# tournament selection
def selection(pop, scores, k=3):
	# first random selection
	selection_ix = np.random.randint(len(pop))
	for ix in np.random.randint(0, len(pop), k-1):
		# check if better (e.g. perform a tournament)
		if scores[ix] > scores[selection_ix]:
			selection_ix = ix
	return pop[selection_ix]

In [12]:
# crossover two parents to create two children
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if np.random.rand() < r_cross:
		# select crossover point that is not on the end of the string
		pt = np.random.randint(1, len(p1)-2)
		# perform crossover
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

In [13]:
# mutation operator
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if np.random.rand() < r_mut:
			# flip the bit
			bitstring[i] = 1 - bitstring[i]

In [14]:
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from imblearn.metrics import specificity_score

def obj(popi, data):
  #print(popi)
  #print(data.shape)
  dtr=[]
  #pd.DataFrame(columns=data.columns)
  dts=[]
  #pd.DataFrame(columns = data.columns)
  for i in range(len(popi)):
    if popi[i]==1:
      #print("data.loc[i]", data.loc[i])
      dtr.append(data.loc[i].values)
    else:
      dts.append(data.loc[i].values)
  dts=pd.DataFrame(dts, columns=data.columns)
  dtr=pd.DataFrame(dtr,columns=data.columns)
  X_train=dtr.iloc[:,:-1].values
  y_train=dtr.iloc[:,-1].values
  X_test=dts.iloc[:,:-1].values
  y_test=dts.iloc[:,-1].values
  # fit model no training data
  model = GaussianNB()
  model.fit(X_train, y_train)
  # make predictions for test data
  y_pred = model.predict(X_test)
  predictions = [round(value) for value in y_pred]
  recall = recall_score(y_test, predictions)
  specificity=specificity_score(y_test, predictions)
  fitness=recall
  return fitness


In [15]:
import numpy as np
import time
#from geneticalgorithm import geneticalgorithm as ga

def GAds_C2(obj, data, n_pop, n_iter, r_cross, r_mut):
  #print(data.shape)
	# initial population of random bitstring
  t0=time.time()
  n_bits=data.shape[0] # the encoding scheme each row will receive 0 or 1
# train  >> test: create two lists 1st list of size= 0.5+ randnumber in[0,1] of ones for train 2nd list of 1-size of zeros

  #pop=[np.random.randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
  #pt=0.5+np.random.rand()/2
  #ps=1-pt
  pop=[np.random.choice(np.arange(0, 2),n_bits, p=[0.3,0.7]).tolist() for _ in range(n_pop)]
	# keep track of best solution
  best, best_eval = 0, obj(pop[0], data)
	# enumerate generations
  for gen in range(n_iter):
    scores = [obj(c,data) for c in pop]
		# check for new best solution
    for i in range(n_pop):
      print("n_iter = ", gen, "n_pop = ", i)
      if scores[i] > best_eval:
        best, best_eval = pop[i], scores[i]

        print(">%d, new best f(%s) = %.3f" % (gen,  pop[i], scores[i]))
    selected = [selection(pop, scores) for _ in range(n_pop)]
		# create the next generation
    children = list()
    for i in range(0, n_pop, 2):
      p1, p2 = selected[i], selected[i-1]
      for c in crossover(p1, p2, r_cross):
        mutation(c, r_mut)
        children.append(c)
		# replace population
    pop = children
  t1=time.time()
  t_exec=t1-t0
  return [best, t_exec,best_eval]

In [16]:
# EX1
n_pop= 50
r_cross=0.5
r_mut=0.5
n_iter=120
GAds_C2(obj, data , n_pop, n_iter, r_cross, r_mut)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
n_iter =  20 n_pop =  0
n_iter =  20 n_pop =  1
n_iter =  20 n_pop =  2
n_iter =  20 n_pop =  3
n_iter =  20 n_pop =  4
n_iter =  20 n_pop =  5
n_iter =  20 n_pop =  6
n_iter =  20 n_pop =  7
n_iter =  20 n_pop =  8
n_iter =  20 n_pop =  9
n_iter =  20 n_pop =  10
n_iter =  20 n_pop =  11
n_iter =  20 n_pop =  12
n_iter =  20 n_pop =  13
n_iter =  20 n_pop =  14
n_iter =  20 n_pop =  15
n_iter =  20 n_pop =  16
n_iter =  20 n_pop =  17
n_iter =  20 n_pop =  18
n_iter =  20 n_pop =  19
n_iter =  20 n_pop =  20
n_iter =  20 n_pop =  21
n_iter =  20 n_pop =  22
n_iter =  20 n_pop =  23
n_iter =  20 n_pop =  24
n_iter =  20 n_pop =  25
n_iter =  20 n_pop =  26
n_iter =  20 n_pop =  27
n_iter =  20 n_pop =  28
n_iter =  20 n_pop =  29
n_iter =  20 n_pop =  30
n_iter =  20 n_pop =  31
n_iter =  20 n_pop =  32
n_iter =  20 n_pop =  33
n_iter =  20 n_pop =  34
n_iter =  20 n_pop =  35
n_iter =  20 n_pop =  36
n_iter 

[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,


In [17]:
# EX2
n_pop= 50
r_cross=0.5
r_mut=0.7
n_iter=120
GAds_C2(obj, data , n_pop, n_iter, r_cross, r_mut)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
n_iter =  20 n_pop =  0
n_iter =  20 n_pop =  1
n_iter =  20 n_pop =  2
n_iter =  20 n_pop =  3
n_iter =  20 n_pop =  4
n_iter =  20 n_pop =  5
n_iter =  20 n_pop =  6
n_iter =  20 n_pop =  7
n_iter =  20 n_pop =  8
n_iter =  20 n_pop =  9
n_iter =  20 n_pop =  10
n_iter =  20 n_pop =  11
n_iter =  20 n_pop =  12
n_iter =  20 n_pop =  13
n_iter =  20 n_pop =  14
n_iter =  20 n_pop =  15
n_iter =  20 n_pop =  16
n_iter =  20 n_pop =  17
n_iter =  20 n_pop =  18
n_iter =  20 n_pop =  19
n_iter =  20 n_pop =  20
n_iter =  20 n_pop =  21
n_iter =  20 n_pop =  22
n_iter =  20 n_pop =  23
n_iter =  20 n_pop =  24
n_iter =  20 n_pop =  25
n_iter =  20 n_pop =  26
n_iter =  20 n_pop =  27
n_iter =  20 n_pop =  28
n_iter =  20 n_pop =  29
n_iter =  20 n_pop =  30
n_iter =  20 n_pop =  31
n_iter =  20 n_pop =  32
n_iter =  20 n_pop =  33
n_iter =  20 n_pop =  34
n_iter =  20 n_pop =  35
n_iter =  20 n_pop =  36
n_iter 

[[1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,


In [18]:
# EX3
n_pop= 100
r_cross=0.5
r_mut=0.2
n_iter=150
GAds_C2(obj, data , n_pop, n_iter, r_cross, r_mut)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
n_iter =  100 n_pop =  0
n_iter =  100 n_pop =  1
n_iter =  100 n_pop =  2
n_iter =  100 n_pop =  3
n_iter =  100 n_pop =  4
n_iter =  100 n_pop =  5
n_iter =  100 n_pop =  6
n_iter =  100 n_pop =  7
n_iter =  100 n_pop =  8
n_iter =  100 n_pop =  9
n_iter =  100 n_pop =  10
n_iter =  100 n_pop =  11
n_iter =  100 n_pop =  12
n_iter =  100 n_pop =  13
n_iter =  100 n_pop =  14
n_iter =  100 n_pop =  15
n_iter =  100 n_pop =  16
n_iter =  100 n_pop =  17
n_iter =  100 n_pop =  18
n_iter =  100 n_pop =  19
n_iter =  100 n_pop =  20
n_iter =  100 n_pop =  21
n_iter =  100 n_pop =  22
n_iter =  100 n_pop =  23
n_iter =  100 n_pop =  24
n_iter =  100 n_pop =  25
n_iter =  100 n_pop =  26
n_iter =  100 n_pop =  27
n_iter =  100 n_pop =  28
n_iter =  100 n_pop =  29
n_iter =  100 n_pop =  30
n_iter =  100 n_pop =  31
n_iter =  100 n_pop =  32
n_iter =  100 n_pop =  33
n_iter =  100 n_pop =  34
n_iter =  100 n_pop = 

[0, 1339.0249271392822, 1.0]

In [19]:
# EX4
n_pop= 150
r_cross=0.5
r_mut=0.3
n_iter=150
GAds_C2(obj, data , n_pop, n_iter, r_cross, r_mut)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
n_iter =  116 n_pop =  100
n_iter =  116 n_pop =  101
n_iter =  116 n_pop =  102
n_iter =  116 n_pop =  103
n_iter =  116 n_pop =  104
n_iter =  116 n_pop =  105
n_iter =  116 n_pop =  106
n_iter =  116 n_pop =  107
n_iter =  116 n_pop =  108
n_iter =  116 n_pop =  109
n_iter =  116 n_pop =  110
n_iter =  116 n_pop =  111
n_iter =  116 n_pop =  112
n_iter =  116 n_pop =  113
n_iter =  116 n_pop =  114
n_iter =  116 n_pop =  115
n_iter =  116 n_pop =  116
n_iter =  116 n_pop =  117
n_iter =  116 n_pop =  118
n_iter =  116 n_pop =  119
n_iter =  116 n_pop =  120
n_iter =  116 n_pop =  121
n_iter =  116 n_pop =  122
n_iter =  116 n_pop =  123
n_iter =  116 n_pop =  124
n_iter =  116 n_pop =  125
n_iter =  116 n_pop =  126
n_iter =  116 n_pop =  127
n_iter =  116 n_pop =  128
n_iter =  116 n_pop =  129
n_iter =  116 n_pop =  130
n_iter =  116 n_pop =  131
n_iter =  116 n_pop =  132
n_iter =  116 n_pop =  133
n_it

[[1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  0,


In [20]:
# EX5
n_pop= 100
r_cross=0.5
r_mut=0.3
n_iter=100
GAds_C2(obj, data , n_pop, n_iter, r_cross, r_mut)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
n_iter =  50 n_pop =  0
n_iter =  50 n_pop =  1
n_iter =  50 n_pop =  2
n_iter =  50 n_pop =  3
n_iter =  50 n_pop =  4
n_iter =  50 n_pop =  5
n_iter =  50 n_pop =  6
n_iter =  50 n_pop =  7
n_iter =  50 n_pop =  8
n_iter =  50 n_pop =  9
n_iter =  50 n_pop =  10
n_iter =  50 n_pop =  11
n_iter =  50 n_pop =  12
n_iter =  50 n_pop =  13
n_iter =  50 n_pop =  14
n_iter =  50 n_pop =  15
n_iter =  50 n_pop =  16
n_iter =  50 n_pop =  17
n_iter =  50 n_pop =  18
n_iter =  50 n_pop =  19
n_iter =  50 n_pop =  20
n_iter =  50 n_pop =  21
n_iter =  50 n_pop =  22
n_iter =  50 n_pop =  23
n_iter =  50 n_pop =  24
n_iter =  50 n_pop =  25
n_iter =  50 n_pop =  26
n_iter =  50 n_pop =  27
n_iter =  50 n_pop =  28
n_iter =  50 n_pop =  29
n_iter =  50 n_pop =  30
n_iter =  50 n_pop =  31
n_iter =  50 n_pop =  32
n_iter =  50 n_pop =  33
n_iter =  50 n_pop =  34
n_iter =  50 n_pop =  35
n_iter =  50 n_pop =  36
n_iter 

[[0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,


In [21]:
# EX6
n_pop= 200
r_cross=0.5
r_mut=0.3
n_iter=150
GAds_C2(obj, data , n_pop, n_iter, r_cross, r_mut)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
n_iter =  125 n_pop =  0
n_iter =  125 n_pop =  1
n_iter =  125 n_pop =  2
n_iter =  125 n_pop =  3
n_iter =  125 n_pop =  4
n_iter =  125 n_pop =  5
n_iter =  125 n_pop =  6
n_iter =  125 n_pop =  7
n_iter =  125 n_pop =  8
n_iter =  125 n_pop =  9
n_iter =  125 n_pop =  10
n_iter =  125 n_pop =  11
n_iter =  125 n_pop =  12
n_iter =  125 n_pop =  13
n_iter =  125 n_pop =  14
n_iter =  125 n_pop =  15
n_iter =  125 n_pop =  16
n_iter =  125 n_pop =  17
n_iter =  125 n_pop =  18
n_iter =  125 n_pop =  19
n_iter =  125 n_pop =  20
n_iter =  125 n_pop =  21
n_iter =  125 n_pop =  22
n_iter =  125 n_pop =  23
n_iter =  125 n_pop =  24
n_iter =  125 n_pop =  25
n_iter =  125 n_pop =  26
n_iter =  125 n_pop =  27
n_iter =  125 n_pop =  28
n_iter =  125 n_pop =  29
n_iter =  125 n_pop =  30
n_iter =  125 n_pop =  31
n_iter =  125 n_pop =  32
n_iter =  125 n_pop =  33
n_iter =  125 n_pop =  34
n_iter =  125 n_pop = 

[[1,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,


In [22]:
# EX7
n_pop= 150
r_cross=0.5
r_mut=0.3
n_ite=200
GAds_C2(obj, data , n_pop, n_iter, r_cross, r_mut)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
n_iter =  116 n_pop =  100
n_iter =  116 n_pop =  101
n_iter =  116 n_pop =  102
n_iter =  116 n_pop =  103
n_iter =  116 n_pop =  104
n_iter =  116 n_pop =  105
n_iter =  116 n_pop =  106
n_iter =  116 n_pop =  107
n_iter =  116 n_pop =  108
n_iter =  116 n_pop =  109
n_iter =  116 n_pop =  110
n_iter =  116 n_pop =  111
n_iter =  116 n_pop =  112
n_iter =  116 n_pop =  113
n_iter =  116 n_pop =  114
n_iter =  116 n_pop =  115
n_iter =  116 n_pop =  116
n_iter =  116 n_pop =  117
n_iter =  116 n_pop =  118
n_iter =  116 n_pop =  119
n_iter =  116 n_pop =  120
n_iter =  116 n_pop =  121
n_iter =  116 n_pop =  122
n_iter =  116 n_pop =  123
n_iter =  116 n_pop =  124
n_iter =  116 n_pop =  125
n_iter =  116 n_pop =  126
n_iter =  116 n_pop =  127
n_iter =  116 n_pop =  128
n_iter =  116 n_pop =  129
n_iter =  116 n_pop =  130
n_iter =  116 n_pop =  131
n_iter =  116 n_pop =  132
n_iter =  116 n_pop =  133
n_it

[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [23]:
# EX8
n_pop= 50
r_cross=0.5
r_mut=0.5
n_iter=100
GAds_C2(obj, data , n_pop, n_iter, r_cross, r_mut)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
n_iter =  0 n_pop =  1
n_iter =  0 n_pop =  2
n_iter =  0 n_pop =  3
n_iter =  0 n_pop =  4
n_iter =  0 n_pop =  5
>0, new best f([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,

[[1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
